In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data observations
*Have NaNs:* Age, Fare (some zeros, nans too), Cabin, Embarked
*NOTE:* maybe need to approximate missing values using some other technique, like an additional model?

* (+) Pclass:
  * 1 - 3 number, 1 being the highest
  * Range: 1-3
  * Current approach: normalize by 3
* (+) Name:
  * has person's title, which could be used (Mr, Ms, Mrs, etc.)
  * From title, can infer marital status?
  * Current approach: extract titles, convert them to one-hot, and calculate 'Married' based on title (1 - married (Mr, Mrs), -1 - unmarried (Miss, Master), 0 - unknown (other titles))
  * Potential improvements: use more titles for getting 'married'; use 'maiden name' in calculation of 'married'; use 'nickname' somehow?
* (+) Sex:
  * Either male or female
  * male: 65%, female: 35%
  * Current approach: convert to one-hot.
  * Potential improvements: use 1 and -1 for sexes?
* (+) Age:
  * has fractions if approximated. Has missing values.
  * Range: 0.42-80
  * Current approach: fill NaN with average in group-by Pclass-Sex, but create a column that identifies missing values. Also, normalize by 80.
  * Potential improvements: have a better approximation of age.
* (+) SibSp:
  * how many siblings or spouses on board.
  * Range: 0-8
  * Current approach: normalize by 8.
* (+) Parch:
  * How many parents/children. (can be 0 for babies, if with nannies)
  * Range: 0-6
  * Current approach: normalize by 6.
* Ticket:
  * A number with some optional letters (which can have some meaning?).
  * Has repetitions (maybe for people travelling together).
* (+) Fare:
  * can have zeros (what do they mean?). Can have omitted (just one in test).
  * Range: 0-512.3292
  * Current approach: fill nan with mean, normalize by 512. 
* (+) Cabin:
  * has a lot of omitted values (78%). Can have multiple values (probably for families?).
  * One value is a letter with a number. (both probably have meaning and impact?)
  * Current approach: convert to one-hot (based on letter), include a 'nan' column for those that are missing values. Create a column for cabin number, and a column to identify missing numbers.
  * Potential improvements: maybe cabin number itself doesn't mean much? Also, maybe need to deal with missing values in a different way? Also, maybe deal with multiple values better?
* (+) Embarked:
  * Either of 3 letters (with different frequency). Has just a few omitted.
  * S - 72/65%, C - 19/24%, Q - 9/11%
  * Current approach: convert to one-hot matrix.
  * Potential improvements: somehow take into the account different distribution of embarkation city?

In [4]:
def prepare_data(csv_data):
    # To display all the columns from left to right without breaking into next line.
    pd.set_option('display.width', 1500)

    import re
    
    features = "Pclass Sex SibSp Parch Fare Embarked Name Cabin Age".split()
    
    titles = ['Mr', 'Mrs', 'Miss', 'Master', 'Ms', 'Don', 'Rev', 'Dr', 'Mme', 'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess', 'Jonkheer', 'Dona']
    
    # === Get X - the features. ===

    X = csv_data[features].copy()

    # == Post-process data ==

    if "Pclass" in X:
        X.Pclass = X.Pclass.divide(3)

    if "SibSp" in X:
        X.SibSp = X.SibSp.divide(8)

    if "Parch" in X:
        X.Parch = X.Parch.divide(6)

    if "Fare" in X:
        # Since only a few would miss 'fare' value, it's okay to fill with average.
        X.Fare = X.Fare.fillna(X.Fare.mean()).divide(512)

    if "Embarked" in X:
        X.Embarked = X.Embarked.astype(pd.api.types.CategoricalDtype(categories="C Q S".split()))
        X = pd.get_dummies(X, columns=["Embarked"])

    if "Name" in X:
        X["Title"] = X.Name.apply(lambda name: re.search(", ([\w ]+).", name).group(1))

        # Try to see if the person is married (1), or not (-1), or unknown (0).
        X["Married"] = X.Title.apply(lambda title: 1 if title in "Mrs Mr".split() else -1 if title in "Miss Master".split() else 0)

        # Get dummies for title, but include all possible values, even those not present in current dataset.
        X.Title = X.Title.astype(pd.api.types.CategoricalDtype(categories=titles))
        X = pd.get_dummies(X, columns=["Title"])
        
    if "Cabin" in X:
        X.Cabin = X.Cabin.fillna("-")
        
        X["Cabin_Number"] = X.Cabin.apply(lambda cabin: int(re.search("\w(\d+)", cabin).group(1)) if len(cabin) > 1 else 0)
        # Do some sort of normalization.
        X.Cabin_Number = X.Cabin_Number.divide(200)
        X["Cabin_Number_Missing"] = np.where(X.Cabin_Number == 0, 1, 0)
        
        X.Cabin = X.Cabin.apply(lambda cabin: cabin[:1])
        
        X.Cabin = X.Cabin.astype(pd.api.types.CategoricalDtype(categories=list("ABCDEFGT")))
        X = pd.get_dummies(X, columns=["Cabin"], dummy_na=True)

    if "Age" in X:
        X["Age_Missing"] = np.where(X.Age.isnull(), 1, 0)

        # No need to skip 'nan' for Age when calculating mean, as Pandas does that automatically.
        # 'transform' will go through each group, and fill its nan values with its mean value.
        # Then, all that will be aggregated back into the column, thus replacing nan values with group's mean.
        X["Age"] = X.groupby("Pclass Sex".split())["Age"].transform(lambda x: x.fillna(x.mean()))
        
        X.Age = X.Age.divide(80)

    # Needs to be after 'Age', since age is using original Sex column.
    if "Sex" in X:
        X.Sex = X.Sex.astype(pd.api.types.CategoricalDtype(categories="male female".split()))
        X = pd.get_dummies(X, columns=["Sex"])
        
    # = Remove extra features = 
    remove_features = "Name".split()
    X = X.drop(columns=remove_features)
    
    # === Get Y - the result values. ===
    Y = csv_data.get("Survived")
    
    print(X.head(5))
    print(None if Y is None else Y.head(5))
    
    return np.array(X), np.array(Y)

In [5]:
print("Train data:")
train_data_X, train_data_Y = prepare_data(train_data)

Train data:
     Pclass  SibSp  Parch      Fare     Age  Embarked_C  Embarked_Q  Embarked_S  Married  Title_Mr  ...  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_nan  Age_Missing  Sex_male  Sex_female
0  1.000000  0.125    0.0  0.014160  0.2750           0           0           1        1         1  ...        0        0        0        0        0        0          1            0         1           0
1  0.333333  0.125    0.0  0.139225  0.4750           1           0           0        1         0  ...        1        0        0        0        0        0          0            0         0           1
2  1.000000  0.000    0.0  0.015479  0.3250           0           0           1       -1         0  ...        0        0        0        0        0        0          1            0         0           1
3  0.333333  0.125    0.0  0.103711  0.4375           0           0           1        1         0  ...        1        0        0        0        0        0          0    

In [6]:
print("Test data:")
test_data_X, _ = prepare_data(test_data)

Test data:
     Pclass  SibSp     Parch      Fare      Age  Embarked_C  Embarked_Q  Embarked_S  Married  Title_Mr  ...  Cabin_C  Cabin_D  Cabin_E  Cabin_F  Cabin_G  Cabin_T  Cabin_nan  Age_Missing  Sex_male  Sex_female
0  1.000000  0.000  0.000000  0.015291  0.43125           0           1           0        1         1  ...        0        0        0        0        0        0          1            0         1           0
1  1.000000  0.125  0.000000  0.013672  0.58750           0           0           1        1         0  ...        0        0        0        0        0        0          1            0         0           1
2  0.666667  0.000  0.000000  0.018921  0.77500           0           1           0        1         1  ...        0        0        0        0        0        0          1            0         1           0
3  1.000000  0.000  0.000000  0.016919  0.33750           0           0           1        1         1  ...        0        0        0        0        0     

In [7]:
from sklearn.model_selection import train_test_split

train_X, dev_X, train_Y, dev_Y = train_test_split(train_data_X, train_data_Y, test_size=0.15, random_state=42)

train_Y = train_Y.reshape((-1, 1))
dev_Y = dev_Y.reshape((-1, 1))

print(train_X.shape, train_Y.shape)
print(dev_X.shape, dev_Y.shape)

(757, 41) (757, 1)
(134, 41) (134, 1)


In [8]:
# Uncomment for the submission - use whole data set for training.

# train_X, train_Y = train_data_X, train_data_Y

In [9]:
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.1.0


In [10]:
from keras import Sequential
from keras.layers import Dense, Dropout

def get_model(input_size):
    model = Sequential([
        Dense(64, input_shape=(input_size,), activation="relu"),
        Dropout(0.2),
        Dense(32, activation="relu"),
        Dropout(0.2),
        Dense(16, activation="relu"),
        Dropout(0.2),
        Dense(1, activation="sigmoid")
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(3e-4), metrics=["accuracy"], loss="binary_crossentropy")
    
    return model

Using TensorFlow backend.


In [11]:
model = get_model(train_X.shape[1])

In [12]:
model.fit(train_X, train_Y, epochs=1000, batch_size=10)

Epoch 1/1000
757/757 [==============================] - 1s 787us/step - loss: 0.6216 - accuracy: 0.6896
Epoch 2/1000
757/757 [==============================] - 0s 192us/step - loss: 0.5576 - accuracy: 0.7345
Epoch 3/1000
757/757 [==============================] - 0s 212us/step - loss: 0.5213 - accuracy: 0.7741
Epoch 4/1000
757/757 [==============================] - 0s 192us/step - loss: 0.4919 - accuracy: 0.7939
Epoch 5/1000
757/757 [==============================] - 0s 195us/step - loss: 0.4933 - accuracy: 0.7939
Epoch 6/1000
757/757 [==============================] - 0s 223us/step - loss: 0.4889 - accuracy: 0.7900
Epoch 7/1000
757/757 [==============================] - 0s 203us/step - loss: 0.4808 - accuracy: 0.7886
Epoch 8/1000
757/757 [==============================] - 0s 200us/step - loss: 0.4888 - accuracy: 0.7886
Epoch 9/1000
757/757 [==============================] - 0s 182us/step - loss: 0.4649 - accuracy: 0.8005
Epoch 10/1000
757/757 [==============================] - 0s 185u

In [13]:
_, accuracy = model.evaluate(train_X, train_Y, verbose=2)

print('\nTrain accuracy:', accuracy)


Train accuracy: 0.9128137230873108


In [14]:
_, accuracy = model.evaluate(dev_X, dev_Y, verbose=2)

print('\nDev accuracy:', accuracy)


Dev accuracy: 0.8283582329750061


In [15]:
predictions = model.predict(test_data_X)

predictions = np.round(predictions).astype(np.uint8).reshape((-1))

print(predictions)

[0 0 0 0 0 0 0 0 1 0 0 1 1 0 1 1 0 0 0 1 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 1 0 1 1 0 0 0 1
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 0 1 1 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 1 0
 0 1 1 1 1 0 0 1 0 0 1]


In [16]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('dl_submission.csv', index=False)